# Manipulation de fichier

SparkSQL est capable de lire et écrire des données depuis ou vers des fichiers de différents formats. Ces fichiers peuvent être des fichiers simples, des fichiers compressés, des fichiers partitionnés, des fichiers partitionnés sur HDFS.

Dans ce notebook, nous allons voir comment se comporte SparkSQL avec les fichiers.

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [1]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("SparkSQL - Fichiers")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit des fonctions supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

Compiling /home/jovyan/work/internal/spark_helper.scLoading spark-stubs
Getting spark JARs


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark UI

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.rdd._


spark: SparkSession = org.apache.spark.sql.SparkSession@77361fa
import spark.implicits._

// Ce script fournit des fonctions supplémentaires pour rendre l'affichage plus confortable

import $file.$                      , spark_helper._

Exécutez la cellule ci-dessous à chaque fois que vous souhaitez recommencer les exercices plus bas.

In [ ]:
def cleanTarget = {
    import java.nio.file.{Files, Path, Paths}

    def deleteDirectory(path: Path): Unit = {
      if (Files.isDirectory(path)) {
        // List the directory contents and delete them recursively
        Files.list(path).forEach(deleteDirectory)
      }
      // Delete the file or directory (if it's a directory, it should be empty by now)
      Files.delete(path)
    }

    val targetDirectory = Paths.get("target/")
    deleteDirectory(targetDirectory)
    Files.createDirectory(targetDirectory)
}

cleanTarget

## Lecture d'un fichier JSON compressé

SparkSQL est capable de gérer naturellement les fichiers compressés, sur différents algorithmes de compression (gzip, bzip2, snappy, lz4, zstd...). La compression permet de gagner de l'espace de stockage et d'augmenter le débit du transfert de données. Il sera en général plus efficace sur les fichiers textes que sur les fichiers binaires. La compression demande un peu plus d'utilisation CPU.

Avec la commande shell ci-dessous, nous pouvons voir qu'il existe un JSON compressé.

In [ ]:
shell("ls -algFh data/")

Utilisez Spark pour charger le fichier JSON compressé.

La méthode `.repartition()` plus bas permet de forcer la redistribution des données dans plusieurs partitions. La valeur passée en paramètre correspond au nombre de partitions souhaité. Cette valeur est limitée par le nombre de Core/CPU disponibles.

In [ ]:
val rawDataframe = ???

val dataframe =
  rawDataframe
    .repartition(4)

dataframe.showHTML(limit=10, truncate=40)

Nous allons voir combien de partitions sont associées au dataframe.

In [ ]:
dataframe.rdd.getNumPartitions

## Sauvegarde dans des fichiers Parquet

Nous allons maintenant tester différents algorithmes de compression.

La fonction ci-dessous va permettre de visualiser pour chaque algorithme ses performances et termes de capacité de compression.

In [ ]:
def testSaveParquet(dataframe: DataFrame, alg: String): Unit = {
  val file = s"orders-$alg.parquet"
  dataframe.repartition(8).write.option("compression", alg).parquet(s"target/$file")
  shell(s"ls -algFh target/$file")
  shell(s"du -h target/$file")
}

Dans chaque cas ci-dessous, regardez et comparez les différents résultats obtenus.

### Pas de compression

In [ ]:
testSaveParquet("none")

### Snappy compression

In [ ]:
testSaveParquet("snappy")

### GZip

In [ ]:
testSaveParquet("gzip")

### BZip2

In [ ]:
testSaveParquet("bzip2")